In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from datetime import datetime as dt
import pandas as pd
from db.db_operations import execute_db_operations

from generators.full_generators import (
    create_company_data,
    create_all_erp_data,
    create_mapping_between_all, 
)

company_name = "Lego"

In [2]:
data = create_company_data(company_name=company_name, save_to_csv=True)

Generating data for company: Lego...
Context data for Lego generated: {'company_name': 'Lego', 'count_employee': 350, 'count_department': 20, 'count_customer': 55, 'count_product': 180, 'count_procurement': 120, 'count_service': 100, 'count_account': 60, 'count_vendor': 65, 'estimated_product': 400000000, 'estimated_service': 80000000, 'estimated_overhead': 442000000, 'estimated_revenue': 649000000}
Generating dimensions for company: Lego.
This usually takes 2-5 mins.

=== Attempt 1 ===
✔ Roles and Names generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Customers data generated.
✔ Departments data generated.
✔ Vendors data generated.
✔ Business Units and company data generated.
✔ Payroll data generated.
✔ All table lengths validated.
✔ All CSVs saved to: data/outputdata

 * Semantic mapping started * :
Time estimate: 3-5 minutes
✔ Procurement mapping done!
✔ Service mapping done!
⚠️ No intercompany candidates found → skipping intercompany 

In [52]:
gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")

C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_33460\3585764430.py:1: DtypeWarning: Columns (0,1,6,7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")


In [53]:
gl_actual = gl[gl["version"] == "Actual"]
gl_actual

,document_number,debit_credit,date,amount,quantity,account_id,product_id,procurement_id,service_id,bu_id,vendor_id,customer_id,version
0,78001011,NaN,2024-08-04,-183901.45,14.0,Retail COS,NaN,ABS resin pellets,NaN,LEGO_FR_RETAIL,Covestro Plastics,NaN,Actual
1,78001139,NaN,2024-09-08,-183956.16,7.0,Retail COS,NaN,ABS resin pellets,NaN,LEGO_FR_RETAIL,Covestro Plastics,NaN,Actual
2,78001077,NaN,2024-03-02,-183942.23,23.0,Retail COS,NaN,ABS resin pellets,NaN,LEGO_FR_RETAIL,Covestro Plastics,NaN,Actual
3,78001028,NaN,2024-09-13,-183986.42,11.0,Retail COS,NaN,ABS resin pellets,NaN,LEGO_FR_RETAIL,Covestro Plastics,NaN,Actual
4,78001175,NaN,2024-11-06,-184017.10,6.0,Retail COS,NaN,ABS resin pellets,NaN,LEGO_FR_RETAIL,Covestro Plastics,NaN,Actual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
589587,78002366,NaN,2024-04-18,4865.59,6.0,Retail Gross Sales,LEGO Technic Airplanes,NaN,NaN,LEGO_FR_RETAIL,NaN,Target,Actual
589588,78002010,NaN,2024-10-29,4860.53,4.0,Retail Gross Sales,LEGO Technic Airplanes,NaN,NaN,LEGO_FR_RETAIL,NaN,Target,Actual
589589,78002308,NaN,2025-04-20,3242.77,3.0,Retail Gross Sales,LEGO Technic Airplanes,NaN,NaN,LEGO_FR_RETAIL,NaN,Target,Actual
589590,78002221,NaN,2024-05-12,3244.94,3.0,Retail Gross Sales,LEGO Technic Airplanes,NaN,NaN,LEGO_FR_RETAIL,NaN,Target,Actual


In [54]:
gl_actual.drop(columns=["version", "debit_credit"], inplace=True)

C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_33460\2222509610.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gl_actual.drop(columns=["version", "debit_credit"], inplace=True)


In [84]:
gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")
gl_actual = gl[gl["version"] == "Actual"]

gl_actual.drop(columns=["version", "debit_credit"], inplace=True)

df_service = pd.read_csv(f"data/outputdata/dimensions/service.csv")
df_bu = pd.read_csv(f"data/outputdata/dimensions/bu.csv")
df_vendor = pd.read_csv(f"data/outputdata/dimensions/vendor.csv")
df_customer = pd.read_csv(f"data/outputdata/dimensions/customer.csv")
df_account = pd.read_csv(f"data/outputdata/dimensions/account.csv")
df_procurement = pd.read_csv(f"data/outputdata/dimensions/procurement.csv")
df_products = pd.read_csv(f"data/outputdata/dimensions/product.csv")

gl_actual = gl_actual.merge(df_bu[["name", "companyname", "companycode"]], left_on="bu_id", right_on="name", how="left")
gl_actual = gl_actual.merge(df_vendor[["name", "vendor_id"]], left_on="vendor_id", right_on="name", how="left")
gl_actual = gl_actual.merge(df_customer[["name", "customer_id"]], left_on="customer_id", right_on="name", how="left")
df_account.rename(columns={"name": "account_name"}, inplace=True)
gl_actual = gl_actual.merge(df_account[["account_name", "AccountKey"]], left_on="account_id", right_on="account_name", how="left")
gl_actual["amount"] = gl_actual["amount"].astype(float)*10
gl_actual.drop(columns=["account_id", "name_x", "name_y", "name",], inplace=True)
gl_actual.rename(columns={"product_id": "product_name", "service_id": "service_name", "procurement_id": "procurement_name",
                          "vendor_id_x": "vendor_name", "customer_id_x": "customer_name",
                          "vendor_id_y": "vendor_id", "customer_id_y": "customer_id", 
                          "companyname": "company_name", "companycode": "company_code",
                          "AccountKey": "account_id", "bu_id": "business_unit", "ammount": "amount_DKK"
                          }, inplace=True)
reordered_columns = ["date", "company_name", "company_code", "business_unit", "account_id", "account_name",
                     "vendor_id", "vendor_name", "customer_id", "customer_name",
                     "product_name", "service_name", "procurement_name"]
gl_actual = gl_actual[reordered_columns + [col for col in gl_actual.columns if col not in reordered_columns]]
gl_actual.sort_values(by=["date"], inplace=True)
gl_actual

C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_33460\2957253624.py:1: DtypeWarning: Columns (0,1,6,7,8,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  gl = pd.read_csv(f"data/outputdata/fact/general_ledger.csv")
C:\Users\jsteensgaard\AppData\Local\Temp\ipykernel_33460\2957253624.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gl_actual.drop(columns=["version", "debit_credit"], inplace=True)


,date,company_name,company_code,business_unit,account_id,account_name,vendor_id,vendor_name,customer_id,customer_name,product_name,service_name,procurement_name,document_number,amount,quantity
517104,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4001,Retail Gross Sales,NaN,NaN,10.0,Target,LEGO Storage Boxes,NaN,NaN,78002109,3424.6,3.0
462019,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,64.0,Staples Solutions DK,NaN,NaN,NaN,NaN,Packing tape (roll),78001190,-485.1,18.0
7397,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,67.0,Rubberstamp Company DK,NaN,NaN,NaN,NaN,Custom print labels (LEGO logo),78001133,-119.0,7.0
550755,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4001,Retail Gross Sales,NaN,NaN,10.0,Target,LEGO Technic Pull-Back Cars,NaN,NaN,78002287,28552.4,14.0
53009,2024-01-01,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,67.0,Rubberstamp Company DK,NaN,NaN,NaN,NaN,Custom print labels (LEGO logo),78001110,-56.1,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407685,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,64.0,Staples Solutions DK,NaN,NaN,NaN,NaN,Sticky notes (pack),78001172,-96.4,4.0
237498,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,63.0,Lyreco Denmark,NaN,NaN,NaN,NaN,Dust masks (FFP2),78001006,-34.6,3.0
191190,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,63.0,Lyreco Denmark,NaN,NaN,NaN,NaN,Disposable nitrile gloves (industrial),78001030,-53.9,6.0
66721,2025-12-31,Lego France,1003,LEGO_FR_RETAIL,4003,Retail COS,67.0,Rubberstamp Company DK,NaN,NaN,NaN,NaN,Custom print labels (LEGO logo),78001096,-55.2,5.0


In [83]:
UNIQUE_COMPANY_NAMES = gl_actual["company_name"].unique().tolist()
print(UNIQUE_COMPANY_NAMES)

['Lego France', 'Lego Germany', 'Lego Denmark', 'Lego Sweden']


In [3]:
now = dt.now()
version_tag = (
    company_name.lower() 
    + now.strftime("%d") + "/" + now.strftime("%m") + ". Kl: " + now.strftime("%H") + ":" + now.strftime("%M")
)
execute_db_operations(version_tag)

[INFO] Inserting 20 rows into dim_department using to_sql...
[INFO] Inserting 59 rows into dim_customer using to_sql...
[INFO] Inserting 150 rows into dim_product using to_sql...
[INFO] Inserting 109 rows into dim_account using to_sql...
[INFO] Inserting 120 rows into dim_procurement using to_sql...
[INFO] Inserting 100 rows into dim_service using to_sql...
[INFO] Inserting 14 rows into dim_line using to_sql...
[INFO] Inserting 69 rows into dim_vendor using to_sql...
[INFO] Inserting 9 rows into dim_bu using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 400 rows into dim_employee using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)


[INFO] Inserting 134400 rows into fact_payroll using to_sql...


c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:189: DtypeWarning: Columns (0,1,6,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_erp = pd.read_csv("data/outputdata/fact/general_ledger.csv")
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace(mapping)
c:\Users\jsteensgaard\OneDrive - KPMG\Documents\Synthetic_Data_Generation\db\db_operations.py:61: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_opti

[INFO] Inserting 232327 rows into fact_general_ledger using to_sql...
[INFO] Inserting 1050 rows into fact_reporting using to_sql...
